In [48]:
from bertviz import head_view
import torch
from medbert.features.mlm_plos_dataset import MLM_PLOS_Dataset
from medbert.features.mlm_plos_dataset import PatientDatum
from medbert.features.embeddings import BertEmbeddings
from medbert.common import pytorch
import numpy as np
from bertviz import model_view
import importlib
from medbert.features import mlm_plos_dataset
importlib.reload(mlm_plos_dataset)

<module 'medbert.features.mlm_plos_dataset' from 'd:\\phd\\pipelines\\med-bert\\medbert\\features\\mlm_plos_dataset.py'>

In [49]:
inputs = torch.load("..\\data\\tokenized\\example_data.pt")
vocab = torch.load("..\\data\\vocab\\example_data.pt")
inv_vocab = {v:k for k,v in vocab.items()}
model = torch.load("..\\models\\mlm_plos_pretrained\\test\\model.pt")
model.eval()
patient_num = 8
max_len = 50
inputs = {k: v[patient_num:patient_num+2] for k, v in inputs.items()}
#dataset = MLM_PLOS_Dataset(inputs, vocab, max_len=max_len)
dataset = PatientDatum(inputs, vocab, )


In [50]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)  # type: ignore

input = [d for d in dataloader][0]#[d for i,d in enumerate(dataloader) if i<20][patient_num]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
config = model.config
embeddings = BertEmbeddings(config=config)
# input = pytorch.batch_to_device(input, device)
embedding_output = embeddings(input['codes'], input['segments'])
# process
number_codes = input['attention_mask'].sum()
outputs = model(inputs_embeds=embedding_output, 
            attention_mask=input['attention_mask'], 
            labels=input['labels'],
            next_sentence_label=input['plos'],
            output_attentions=True) 
attention = outputs[-1] 
attention_new = []
for att in attention:
    att = att[:,:,:number_codes,:number_codes]
    attention_new.append(att)
attention_new = tuple(attention_new)
# attention = truncate_attention(attention, input['codes'])
input_tokens = [inv_vocab[i] for i in inputs['codes'][0]]
head_view(attention_new, input_tokens)

<IPython.core.display.Javascript object>